# Assignment 8: Document Retrieval

In this assignment, you're going to explore various similarity metrics we have learned so far and predict the label of a text document by assigning the label of its nearest neighbor to it. Due to the sparseness of document vectors (lots of zero entries), which results in a significantly large computational runtime, we will apply a method called Random Projection to reduce the dimensions of each document vector.

Fill in the cells provided marked `TODO` with code to answer the questions.

> Copyright ©2022 Pemi Nguyen.  All rights reserved.  Permission is hereby granted to students registered for University of Washington CSE/STAT 416 for use solely during Spring Quarter 2022 for purposes of the course.  No other use, copying, distribution, or modification is permitted without prior written consent. Copyrights for third-party components of this work must be honored.  Instructors interested in reusing these course materials should contact the author.

# Part 1: Similarity Metrics

In the first part, you will look at various metrics to measure similarity between text documents and think about their relative merits.

The file `data.csv` contains information about 1000 articles. Each line in the file contains three fields: `(articleID, wordID, Count)` representing the number of times that word `wordID` occurs in article `articleID`. We don't store words that have zero frequencies in each article. No specific word names or article names are provided.

The file `labels.csv` contains the `groupID`, the newsgroup that a given `articleID` belongs to (for a total of 1000 articles). The file `groups.csv` contains the specific name of each `groupID` (for a total of 20 newsgroups).

## Load the datasets

In [21]:
import pandas as pd
data = pd.read_csv('data.csv')
groups = pd.read_csv('groups.csv')
labels = pd.read_csv('labels.csv')

`data.csv` stores the non-zero frequencies of all words in each article.

In [22]:
data

`groups.csv` stores the names of 20 newsgroups.

In [23]:
groups

`labels.csv` stores the newsgroup that each article belongs to.

In [24]:
labels

## Preprocessing the data

Create an original matrix using the bag-of-words approach (since we stores the frequencies of words in each article). `scipy` provides tools for creating sparse matrices using multiple data structures, as well as tools for converting a dense matrix to a sparse matrix.

In [25]:
import scipy.sparse as sparse
original_matrix = sparse.csr_matrix((data.Count, (data.articleID, data.wordID))).toarray()

There are 1,000 articles and 61,067 words in the bag of words.

In [26]:
original_matrix.shape

We call this matrix sparse because it has a lot of zeroes in it.

In [27]:
original_matrix[0]

In [28]:
import numpy as np

# Set seed for the code
np.random.seed(416)

num_groups = len(groups)

After , we're ready to use the new matrix where each document is represented by a 1000-dimensional vector.

## Calcutating various similarity metrics

We have learned four similarity metrics so far: Jaccard, Euclidean, Manhattan and Cosine. In Machine Learning, it's important to decide which metric you should use for your pipeline, but it's difficult to understand what these metrics do without visualizing its effects on our data.

Note that Jaccard and Cosine similarity are numbers between $0$ and $1$, where as Manhattan and Euclidean between $-\infty$ and $0$ (with higher numbers indicating more similarity).

For questions 1-4, on EdStem, we will test your calculations on the following two vectors:

```
x = [1.4, 4.3, 2.5, 8.39]
y = [17.3, 0.32, 9.21, 1.12]
```

### 🔍 **Question 1**: Calculate Jaccard similarity:

The Jaccard similarity between two vector $x$ and $y$ is defined as:
$$J(x, y) = \frac{\sum_i \min(x_i, y_i)}{\sum_i \max(x_i, y_i)}$$

Hint: To use `numpy` for this, you can concatenate these two arrays using np.concatenate  `np.vstack((x, y))`, then use `np.min(axis=0)`, `np.max(axis=0)` and `np.sum`.

In [29]:
### edTest(test_q1_jaccard) ###

def jaccard_similarity(x, y):
    """
    Calculate the Jaccard similarity between two vectors x and y

    Args:
    - x (np.array): The first vector
    - y (np.array): The second vector

    Returns:
    - the Jaccard similarity between two vectors x and y
    """
    # TODO
    inter = np.vstack((x,y))
    j = (np.sum(np.min(inter, axis=0)))/(np.sum(np.max(inter,axis=0)))
    return j

### 🔍 **Question 2**: Calculate Euclidean similarity:

The Euclidean similarity between two vector $x$ and $y$ is defined as:
$$L_2(x, y) = - \left\| x- y\right\|_2 = - \sqrt{\sum_i (x_i - y_i)^2}$$

Note: Euclidean similarity has a $-$ sign to indicate similarity (which is the opposite of Euclidean distance).

Hint: Look into `np.linalg.norm`.

In [30]:
### edTest(test_q2_euclidean) ###

def euclidean_similarity(x, y):
    """
    Calculate the Euclidean similarity between two vectors x and y

    Args:
    - x (np.array): The first vector
    - y (np.array): The second vector

    Returns:
    - the Euclidean similarity between two vectors x and y
    """
    # TODO
    e_similarity = -np.linalg.norm(x-y)
    return e_similarity

### 🔍 **Question 3**: Calculate Manhattan similarity:

The Manhattan similarity between two vector $x$ and $y$ is defined as:
$$L_1(x, y) = - \left\| x- y\right\|_1 = - \sum_i |x_i - y_i|$$

Note: Same thing as Euclidean similarity, Manhattan has a $-$ sign to indicate similarity (which is the opposite of Manhattan distance).

Hint: Look into `np.linalg.norm`.

In [31]:
### edTest(test_q3_manhattan) ###

def manhattan_similarity(x, y):
    """
    Calculate the Manhattan similarity between two vectors x and y

    Args:
    - x (np.array): The first vector
    - y (np.array): The second vector

    Returns:
    - the Manhattan similarity between two vectors x and y
    """
    # TODO
    man = sum(-abs(val1-val2) for val1, val2 in zip(x,y))
    return man

### 🔍 **Question 4**: Calculate Cosine similarity:

The Cosine similarity between two vector $x$ and $y$ is defined as:
$$S_c(x, y) = \frac{\sum_i x_i \cdot y_i}{\left\| x\right\|_2 \left\| y\right\|_2}$$

Hint: Look into `np.dot` (dot product between two vectors) and `np.linalg.norm`.

In [32]:
### edTest(test_q4_cosine) ###

def cosine_similarity(x, y):
    """
    Calculates the Cosine similarity between two vectors x and y
    
    Args:
    - x (np.array): The first vector
    - y (np.array): The second vector

    Returns:
    - the Cosine similarity between two vectors x and y
    """
    # TODO
    from numpy.linalg import norm
    cos = np.dot(x,y)/(norm(x)*norm(y))
    return(cos)

## Visualizing group similarities and neartest neighbor document classification accuracies using a heatmap.

For each metric, we will visualize group similarities and neartest neighbor document classification accuracies with a heatmap. The plot will look like a $20 \cdot 20$ matrix. Rows and columns are index by newsgroups
(in the same order). A darker color indicates more similarity, and vice-versa.

In [33]:
import matplotlib.pyplot as plt
import warnings

# Heatmap
def makeHeatMap(data, names, color, similarity_metric):
#to catch "falling back to Agg" warning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        #code source: http://stackoverflow.com/questions/14391959/heatmap-in-matplotlib-with-pcolor
        fig, ax = plt.subplots()
        #create the map w/ color bar legend
        heatmap = ax.pcolor(data, cmap=color)
        cbar = plt.colorbar(heatmap)

        # Set the size of the plots
        fig = plt.gcf()
        fig.set_size_inches(14, 11)

        # put the major ticks at the middle of each cell
        ax.set_xticks(np.arange(data.shape[0])+0.5, minor=False)
        ax.set_yticks(np.arange(data.shape[1])+0.5, minor=False)

        # want a more natural, table-like display
        ax.invert_yaxis()
        ax.xaxis.tick_top()
        plt.xticks(rotation=90)

        ax.set_xticklabels(names)
        ax.set_yticklabels(names)
        ax.set_title(similarity_metric)

        plt.tight_layout()

        # plt.savefig(outputFileName, format = 'png')
        plt.show()
        plt.close()

## Calculating similaritities between every two newsgroups

*Optional: This part takes 15 minutes to run, so you are not required to run it. Due to EdStem's limited computational limits, you cannot run the following cell. You can try running this notebook locally, or upload it on Google Colab for more computational usage.* 

We calculate the pairwise similarities between every two articles based on these 4 above metrics. After that, we then calculate pairwise similarities between every two newsgroup A and newsgroup B by averaging the similarity results between each article in A with each article in B.

If you implement the similarity metric calculations correctly, here are the results:

![alt text](https://drive.google.com/uc?export=view&id=1QAUmPFLnCtzWNFRt7piq2Wy-E5cIC-lA)

![alt text](https://drive.google.com/uc?export=view&id=16eBriprMF2-4naTc8PIbwJbchpv6L3Uh)

![alt text](https://drive.google.com/uc?export=view&id=1cI5uaWTbjogiT9rGeVP6pjNFQJBzNhYJ)

![alt text](https://drive.google.com/uc?export=view&id=1Ukag4wE8oHE0oKyCByX0gnua7dDpuNK9)


Uncomment the code (using `Cmd + /` if you're on a MacBook) to try running it.

In [34]:
# import itertools

# similarity_matrix = np.zeros((4, num_groups, num_groups))
# for i,j in itertools.combinations_with_replacement(groups.index, 2):
#     articles_i = labels[labels.label == i].index
#     articles_j = labels[labels.label == j].index
#     pairwise_results = np.zeros((4, len(articles_i)*len(articles_j)))
#     k = 0
#     for a,b in itertools.product(articles_i, articles_j):
#         pairwise_results[0, k] = jaccard_similarity(original_matrix[a], original_matrix[b])
#         pairwise_results[1, k] = euclidean_similarity(original_matrix[a], original_matrix[b])
#         pairwise_results[2, k] = manhattan_similarity(original_matrix[a], original_matrix[b])
#         pairwise_results[3, k] = cosine_similarity(original_matrix[a], original_matrix[b])
#         k += 1
#     similarity_matrix[:,i,j] = similarity_matrix[:,j,i] = pairwise_results.mean(axis=1)

# makeHeatMap(similarity_matrix[0], groups.group_name, 'Blues', 'Jaccard Similarity between 20 newsgroups')
# makeHeatMap(similarity_matrix[1], groups.group_name, 'Blues', 'Euclidean Similarity between 20 newsgroups')
# makeHeatMap(similarity_matrix[2], groups.group_name, 'Blues', 'Manhattan Similarity between 20 newsgroups')
# makeHeatMap(similarity_matrix[3], groups.group_name, 'Blues', 'Cosine Similarity between 20 newsgroups')

# Part 2: Nearest neighbor classification

A nearest-neighbor classification system is conceptually extremely simple, and often is very effective. Given a large dataset of labeled examples, a nearest-neighbor classification system will predict a label for a new
example, $x$, as follows: It will find the element of the labeled dataset that is closest to $x$ — closest in whatever metric makes the most sense for that dataset — and then output the label of this closest point.

## Comparing between above 4 metrics

Of the 4 metrics above, it seems like Manhattan and Euclidean metrics are pretty off, since the heatmap is dark at the majority of cells, which suggests a lot of these 20 different newsgroups are pretty similar to each other (which doesn't make sense). Besides, the fact that the range of these two metrics are unbounded ($-\infty$ to $0$) is also a disadvantage.

Both Jaccard and Cosine heatmaps are more nuanced, since their range is bounded (between 0 and 1). Looking at their heatmaps, we can see there is a wide distribution of color shades from white to dark blue across different cells. Between these two, Cosine seems better because the middle range of values on the legends for Jaccard is quite limited (0.05 to 0.13), but that for Cosine is wider (0.20 to 0.50).

As a result, for this problem, we will choose Cosine Similarity metric for our nearest neighbor classification task. In general, in NLP, to measure similarity between document vectors, Cosine similarity is more preferred because of another nice interpretation: It represents the angle between two vectors.

## Nearest neighbor search

Let's implement our nearest neighbor search algorithm. From a vector that represents an article, we will find another article that has the highest similarity to it.

In [35]:
def nearest_neighbor_search(article_vector, data_matrix, articleID, metric=cosine_similarity):
    """
    For each article, return the articleID of the article most similar to it (excluding itself)
    according to a similarity metric
    
    Args:
    - article_vector (np.array): A numpy array that stores the vector for an article
    - data_matrix (np.array): A numpy array which is the matrix representation for all articles
    - articleID (int): The articleID of an article that we are querying for its nearest neighbor.
    - metric (func): The function of a similarity metric (from 4 choices above). 
    By default, we're choosing cosine_similarity. You can also choose manhattan_similarity, euclidean_similarity
    and jaccard_similarity.

    Returns:
    - The articleID of the nearest neighbor article.

    """
    # Calculating the similarity score between a specific article and all the articles
    similarity_vec = np.apply_along_axis(lambda row: metric(row, article_vector), 1, data_matrix)

    # Set similarity result at the index of the current article to be -inf to avoid returning it
    similarity_vec[articleID] = np.NINF

    return np.nanargmax(similarity_vec)

## Classification Matrix

Build a `classification_matrix` in which the columns represent the true labels and the columns represent predictions. A cell value at index `(i, j)`, particularly `classification_matrix[i, j]`, indicates the number of articles which have true labels with group_ID `i` and predictions with group_ID `j`. 

For example, for a matrix:

```
[[3, 2, 0],
[1, 0, 0], 
[4, 0, 0]]
```

At column 0 and row 2, there are 4 articles whose true labels are `groupID = 2` and predictions whose predicted labels are `groupID = 0`

In [36]:
def build_classification_matrix(data_matrix, groups=groups, labels=labels, metric=cosine_similarity):
    """
    Build a classification matrix in which the columns represent the true labels 
    and the columns represent predictions. 
    A cell value at index `(i, j)`, particularly `classification_matrix[i, j]`, 
    indicates the number of articles which have true labels with group_ID `i` 
    and predictions with group_ID `j`. 
    
    Args:
    - data_matrix (np.array): A numpy array which is the matrix representation for all articles
    - groups (pd.DataFrame): A DataFrame storing information about different groups of a dataset
    - labels (pd.DataFrame): A DataFrame storing information about the labels for all articles
    - metric (func): The function of a similarity metric (from 4 choices above). 
    By default, we're choosing cosine_similarity. You can also choose manhattan_similarity, euclidean_similarity
    and jaccard_similarity.

    Returns:
    - A classification matrix

    """
    classification_matrix = np.zeros((len(groups),len(groups)))

    for i in range(len(labels)):
        true_label = labels.loc[i,'label']
        doc = data_matrix[i]
        most_sim_doc = nearest_neighbor_search(doc, data_matrix, i, metric)
        most_sim_label = labels.loc[most_sim_doc,'label']
        classification_matrix[true_label,most_sim_label] += 1
    
    return classification_matrix

## Visualizing classification matrix using original dimensions

*Optional: This part takes 10 minutes to run, so you are not required to run it. Due to EdStem's limited computational limits, you cannot run the following cell. You can try running this notebook locally, or upload it on Google Colab for more computational usage.* 

Here's the heatmap for this matrix if you don't wish to run the code:

![alt text](https://drive.google.com/uc?export=view&id=1Ownsmdy_2QWBB1yG9FzSczlk8Sspgnx4)

In [37]:
# classification_matrix_original = build_classification_matrix(original_matrix)
# makeHeatMap(classification_matrix_original, groups.group_name, 'Blues', 'Classification Matrix with original dimensions')

### 🔍 **Question 5**: Calculate total classfication accuracy

Based on a classification matrix, let's calculate the total classification accuracy.

For example, for this matrix:

```
[[3, 2, 0],
[1, 0, 0],
[4, 0, 0]]
```

the total accuracy is = Total correct predictions / Total number of examples

$$= \frac{3 + 0 + 0}{3 + 2 + 0 + 1 + 0 + 0 + 4 + 0+0} = 0.3$$

In [38]:
### edTest(test_q5_classification_accuracy) ###

def compute_classification_accuracy(classification_matrix):
    """
    Computes classification accuracy from a classification matrix
    
    Args:
    - classification matrix (np.array): A classification matrix

    Returns:
    - The classification accuracy for a classification matrix.
    """
    
    # TODO
    correct = np.sum(np.diag(classification_matrix))
    tot = np.sum(classification_matrix)
    accuracy = correct/tot
    return accuracy

For the classsification matrix with the original dimensions, the total classification accuracy is $0.456$. If you uncomment the above code, uncomment this one below too to see how the code runs.

In [39]:
# compute_classification_accuracy(classification_matrix_original)

## Dimensionality Reduction using Random Projection

It takes lots of computation resources for dealing with 61k dimensions. Aside from PCA, another technique that can help us reduce the dimensions of an original matrix is **Random Projection**.

Random Projection: Given a set of $k$-dimensional vectors ${v_1, v_2, ...}$, define a $d \times  k$ matrix $M$ by drawing each entry randomly (and independently) from a normal distribution of mean $0$ and variance
$1$. The $d$-dimensional reduced vector corresponding to $v_i$
is given by the matrix-vector product $Mv_i$

We can think of the matrix $M$ as a set of $d$ random $k$-dimensional vectors ${w_1,...w_d}$ (the rows of $M$), and then the $j$-th coordinate of the reduced vector $Mv_i$ is the inner product between that $v_i$ and $w_j$.

Try various reduced dimensions $[10, 50, 100, 200, 500]$ to see how the classification accuracy improves and how the heatmap differs.

Note: EdStem's kernel will stop if you choose `k > 500` due to EdStem's computational limitations. 

In [40]:
ks = [10, 50, 100, 200, 500]

classification_matrix_reduced = np.zeros((len(ks), num_groups, num_groups))
classification_accuracies = []

m = original_matrix.shape[1]
for idx, k in enumerate(ks):
    random_matrix = np.random.normal(size=(m,k))
    projected_matrix = original_matrix @ random_matrix
    classification_matrix_reduced[idx] = build_classification_matrix(projected_matrix)
    current_accuracy = compute_classification_accuracy(classification_matrix_reduced[idx])
    classification_accuracies.append(current_accuracy)
    print("Nearest neighbor classification accuracy with k = ", k, ": ", current_accuracy)
    makeHeatMap(classification_matrix_reduced[idx], groups.group_name, 'Blues', 'Dimensionality Reduction Classification (k={})'.format(k))

### Visualizing the improvement of classification accuracy with increasing number of reduced dimensions

You can see from above that using $k = 500$, the accuracy can match closely to the accuracy when using the original number of dimensions $d = 61,067$. As we use more dimensions, the classification accuracy seems to improve. However, there is a clear tradeoff between the amount of improvement and computational efficiency.

In [41]:
plt.plot(ks, classification_accuracies)
plt.xlabel("Number of reduced dimensions")
plt.ylabel("Classification accuracy")

## Visualizing classification matrices using other metrics

Let's see why Cosine similiarity is the best out of all metrics by comparing the classification accuracies with the same number of reduced dimensions.

### Jaccard Similarity

Clearly the accuracies are pretty low here. The heatmaps are also supposed to be dark along the diagonal to indicate high classification accuracies, which isn't the case for Jaccard.

In [0]:
ks = [10, 50, 100]

classification_matrix_reduced = np.zeros((len(ks), num_groups, num_groups))
classification_accuracies = []

m = original_matrix.shape[1]
for idx, k in enumerate(ks):
    random_matrix = np.random.normal(size=(m,k))
    projected_matrix = original_matrix @ random_matrix
    classification_matrix_reduced[idx] = build_classification_matrix(projected_matrix, metric=jaccard_similarity)
    current_accuracy = compute_classification_accuracy(classification_matrix_reduced[idx])
    classification_accuracies.append(current_accuracy)
    print("Nearest neighbor classification accuracy with k = ", k, ": ", current_accuracy)
    makeHeatMap(classification_matrix_reduced[idx], groups.group_name, 'Blues', 'Dimensionality Reduction Classification (k={})'.format(k))

### Manhattan Similarity

In [0]:
ks = [10, 50, 100]

classification_matrix_reduced = np.zeros((len(ks), num_groups, num_groups))
classification_accuracies = []

m = original_matrix.shape[1]
for idx, k in enumerate(ks):
    random_matrix = np.random.normal(size=(m,k))
    projected_matrix = original_matrix @ random_matrix
    classification_matrix_reduced[idx] = build_classification_matrix(projected_matrix, metric=manhattan_similarity)
    current_accuracy = compute_classification_accuracy(classification_matrix_reduced[idx])
    classification_accuracies.append(current_accuracy)
    print("Nearest neighbor classification accuracy with k = ", k, ": ", current_accuracy)
    makeHeatMap(classification_matrix_reduced[idx], groups.group_name, 'Blues', 'Dimensionality Reduction Classification (k={})'.format(k))

### Euclidean Similarity

Both Euclidean and Manhattan similarities have much better accuracies than Jaccard one, but they're not as good as Cosine one for the same number of reduced dimensions.

In [0]:
ks = [10, 50, 100]

classification_matrix_reduced = np.zeros((len(ks), num_groups, num_groups))
classification_accuracies = []

m = original_matrix.shape[1]
for idx, k in enumerate(ks):
    random_matrix = np.random.normal(size=(m,k))
    projected_matrix = original_matrix @ random_matrix
    classification_matrix_reduced[idx] = build_classification_matrix(projected_matrix, metric=euclidean_similarity)
    current_accuracy = compute_classification_accuracy(classification_matrix_reduced[idx])
    classification_accuracies.append(current_accuracy)
    print("Nearest neighbor classification accuracy with k = ", k, ": ", current_accuracy)
    makeHeatMap(classification_matrix_reduced[idx], groups.group_name, 'Blues', 'Dimensionality Reduction Classification (k={})'.format(k))